In [ ]:
## Import packages + libraries for later use
%matplotlib inline

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model

In [ ]:
## Import specific modules needed for CNN
from VisualiseFilters import SaliencyMask, GradientSaliency, VisualBackprop, deprocess_image, vis_img_in_filter, get_layer_names, activation_maps
from Predictions import predictions_rcnn
from ViewHistory import view_history
from GetImages import get_images
from Shuffling import stratified_shuffling_rcnn

In [ ]:
## disp / disp-short / OSNR
data = 'disp'

## Square sides of resolution
img_res = 32

In [ ]:
## File directory to jpg folder
train_dir = 'C:/Users/barry.odonnell/Python/input_eye/train_jpg_files_{}'.format(data)

## Collects training images and test images
train_imgs = ['C:/Users/barry.odonnell/Python/input_eye/train_jpg_files_{}/{}'.format(data, i) for i in os.listdir(train_dir)]

In [ ]:
data = 'disp-short'

In [ ]:
## Obtain image and label arrays
X,y = get_images(train_imgs, img_res, GRAY=True, data=data)

In [ ]:
## Assigns each label to an integer, ex. 12dB == 0, 12.5dB == 1...
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)

## create an array of each value in the set and assign them integers from 0-(n-classes)
Y_train = np.expand_dims(integer_encoded, axis=1)
Y_train = np.asarray(Y_train)
Y = label_encoder.fit_transform(y)

## Convert X,y to numpy arrays to allow more operations to gelp us later
y = np.array(y)

print("Shape of train images is:", X.shape) ## Check for correct shape, should have 4 axes
lb = preprocessing.LabelBinarizer()
lb.fit(Y)

In [ ]:
X_train, Y_train, X_val, Y_val = stratified_shuffling_rcnn(X, Y_train, (max(Y) + 1), 0.3)

In [ ]:
## Begin definition for Neural Network model.
##  A Convolutional Neural Network (CNN) is used for its
##  versatility with images, and it's feature map ability.
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

## Insert image dimensions, preferably square, includinig channel
img_dim = (img_res,img_res,3)

model_name = 'eye_keras_rcnn-model_{}_{}.h5'.format(data, img_res)

## Checks if this model exists.
##  If it does, loads said model and skips model building.
if os.path.exists(model_name):
    print("Loading model...")
    model = load_model(model_name)
    print("{} has been loaded...".format(model_name))
    
## Else, builds 'Sequential' model and will contain the layers stated below 
##  https://keras.io/layers/core/
##  https://keras.io/layers/convolutional/
else:
    print("Building Model...")

    model = Sequential()

    model.add(Conv2D(8, (7,7), input_shape=(img_dim), kernel_initializer='normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(64, (5,5), input_shape=(img_dim), kernel_initializer='normal'))
    model.add(Activation('relu'))

    model.add(Conv2D(16, (5,5), input_shape=(img_dim), kernel_initializer='normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Flatten())
    model.add(Dense(img_res**2))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(1))
    model.add(Activation('relu'))

    model.compile(loss='mean_squared_error',
                  optimizer='adam', 
                  metrics=['mean_squared_error']
                 )

print("\nModel ('{}') built...".format(model_name))

## Returns model info for user to review and check that it is correct
model.summary()

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 0,
    zoom_range = 0,
    width_shift_range = .3,
    height_shift_range = 0,
    horizontal_flip = True,
    vertical_flip = False
    )

datagen.fit(X_train)

In [ ]:
## Begin testing on model
print('Training model...')
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size = 32),
                   epochs = 50,
                   validation_data = (X_val, Y_val),
                   steps_per_epoch = 10,
                   verbose = 1
                   )

print('Validating model...')
mse, acc = model.evaluate(X_val, Y_val, verbose= 1)
print('\nValidation mean-square-error:', mse)


In [ ]:
print("\nSaving model...")
#model.save(model_name) 
(model.save('{0:3.4f}_'.format(mse)+ model_name))
print("\nSaved model as {}...".format(model_name))

In [ ]:
view_history(history, show_acc = False)

In [ ]:
#model = load_model('0.0056_eye_keras_rcnn-model_disp_64.h5')
#model.summary()

In [ ]:
predictions_rcnn(model, X_train, Y_train)

predictions_rcnn(model,
                 X_val,
                 Y_val,
                 convert=True,
                 data=data,
                 types=Y,
                 errorbars=True,
                 mse=mse)

In [ ]:
layers = get_layer_names(model, show=True)

In [ ]:
vis_img_in_filter(model,
                  X_train,
                  img_num=-2,
                  img_res=img_res,
                  layer_num=5,
                  GRAY=True)

In [ ]:
Y_train_label = lb.inverse_transform(Y_train)

activation_maps(model,
                X_train,
                Y_train_label,
                img_res=img_res,
                show_img=4
               )